# Classification II: Logistic Regression

**OBJECTIVES**:

- Differentiate between *Regression* and *Classification* problem settings
- Connect Least Squares methods to Classification through Logistic Regression
- Interpret coefficients of the model in terms of probabilities
- Discuss performance of classification model in terms of accuracy
- Understand the effect of an imbalanced target class on model performance

### Classification Problems as Predicting Categorical Target Feature



In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import statsmodels.api as sm
from scipy import stats

from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_breast_cancer, load_digits, load_iris

### Some example datasets in sklearn for classification
<center>
<img src = https://archive.ics.uci.edu/ml/assets/MLimages/Large14.jpg>
</center>

In [ ]:
cancer = load_breast_cancer()
#print(cancer.DESCR)
#think of the dataset as a class with attributes
#.data
#.target
#.feature_names
X = cancer.data
y = cancer.target

In [ ]:
#create a dataframe
cancer_df = pd.DataFrame(X, columns = cancer.feature_names)
cancer_df['malignant'] = cancer.target
cancer_df.head(2)

In [ ]:
#mean radius by malignance


### Our Motivating Example



In [ ]:
default = pd.read_csv('data/Default.csv', index_col = 0)

In [ ]:
default.info()

In [ ]:
default.head(2)

### Visualizing Default by Continuous Features

In [ ]:
#scatterplot of balance vs. income colored by default status

In [ ]:
#boxplots for balance and income by default


### Considering only `balance` as the predictor



In [ ]:
#create binary default column


In [ ]:
#scatter of Balance vs Default


##### PROBLEM

1. Build a `LinearRegression` model with balance as the predictor.
2. Interpret the $r^2$ score and $rmse$ for your regressor.
3. Predict the default for balances: `[500, 1000, 1500, 2000, 2500]`.  Do these make sense?

### The Sigmoid aka Logistic Function


$$y = \frac{1}{1 + e^{-x}}$$

In [ ]:
#define the logistic


In [ ]:
#domain


In [ ]:
#plot it


### Usage should seem familiar

Fit a `LogisticRegression` estimator from `sklearn` on the features:

```python 
X = default[['balance']]
y = default['binary_default']
```

In [ ]:
#instantiate


In [ ]:
#define X and y


In [ ]:
#train test split


In [ ]:
#fit on the train


In [ ]:
#examine train and test scores


### Similarities to our earlier work

In [ ]:
#there is a coefficient


In [ ]:
#there is an intercept


### Where was the line?

The version of the logistic we have just developed is actually:

$$ y = \frac{e^{ax + b}}{1 + e^{ax + b}} $$

Its output represents probabilities of being labeled the positive class in our example.  This means that we can interpret the output of the above function using our parameters, remembering that we used the `balance` feature to predict `default`.

In [ ]:
def predictor(x):
    line = clf.coef_[0]*x + clf.intercept_
    return np.e**line/(1 + np.e**line)

In [ ]:
#predict 1000


In [ ]:
#predict 2000


In [ ]:
#estimator has this too


### Using Categorical Features

In [ ]:
default.head(2)

In [ ]:
default['student_binary'] = np.where(default.student == 'No', 0, 1)

In [ ]:
X = default['student_binary']

In [ ]:
#instantiate and fit


In [ ]:
#performance


In [ ]:
#coefficients


In [ ]:
#compare probabilities


### Using Multiple Features



In [ ]:
default.columns

In [ ]:
features = ['balance', 'income', 'student_binary']
X = default.loc[:, features]
y = default['binary_default']

**Predictions**:

- student: yes
- balance: 1,500 dollars
- income: 40,000 dollars

- student: no
- balance: 1,500 dollars
- income: 40,000 dollars

### This is similar to our multicollinearity in regression; we will call it confounding

<center>
<img src = 'images/default_confound.png'/>
</center>

In [ ]:
b_sort = default.sort_values(by = 'balance')
students = b_sort.loc[b_sort['student_binary'] == 1]
non_students = b_sort.loc[b_sort['student_binary'] == 0]
num_defaults = b_sort['binary_default'].sum()

In [ ]:
plt.plot(students['balance'], students['binary_default'].cumsum()/students['binary_default'].sum(), label = 'students')
plt.plot(non_students['balance'], non_students['binary_default'].cumsum()/non_students['binary_default'].sum(), label = 'non-students')
plt.title('Confounding in the Default Data')
plt.xlabel('Balance')
plt.ylabel('Default Rate')
plt.grid()
plt.xlim(0, 2300);
plt.legend();
plt.savefig('images/default_confound.png')